# Pandas
<!-- requirement: img/Data_Frame_Data_Series.png -->
<!-- requirement: small_data/fha_by_tract.csv -->
<!-- requirement: small_data/2013_Gaz_tracts_national.tsv -->

Pandas is Python's answer to R.  It's a good tool for small(ish) data analysis -- i.e. when everything fits into memory.

The basic new "noun" in pandas is the **data frame**.

## Nouns (objects) in Pandas

### Data Frames

Like a table, with rows and columns (e.g. as in SQL).  Except:
  - The rows can be indexed by something interesting (there is special support for labels like categorical and timeseries data).  This is especially useful when you have timeseries data with potentially missing data points.
  - Cells can store Python objects. Like in SQL, columns are type homogeneous.
  - Instead of "NULL", the name for a non-existent value is "NA".  Unlike R, Python's data frames only support NAs in columns of some data types (basically: floating point numbers and 'objects') -- but this is mostly a non-issue (because it will "up-cast" integers to float64, etc.)
  
### Data Series:

These are named columns of a DataFrame (more correctly, a dataframe is a dictionary of Series).  The entries of the series have homogenous type.

![img/4-pandas/Data_Frame_Data_Series.png](img/4-pandas/Data_Frame_Data_Series.png)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# a data frame
df1 = pd.DataFrame({
    'number': [1, 2, 3],
    'animal': ['cat', 'dog', 'mouse']
})

df1

,number,animal
0,1,cat
1,2,dog
2,3,mouse


In [5]:
# select the animal series from the dataframe using the different notations

df1['animal']

0      cat
1      dog
2    mouse
Name: animal, dtype: object

In [6]:
## access through point notation
df1.animal

0      cat
1      dog
2    mouse
Name: animal, dtype: object

In [8]:
# show the types of the dataframe columns
type(df1['animal'])

pandas.core.series.Series

In [10]:
type(df1[['animal']])

pandas.core.frame.DataFrame

In [12]:
## cast number to float

df1['number'].astype(float)

0    1.0
1    2.0
2    3.0
Name: number, dtype: float64

In [13]:
df1

,number,animal
0,1,cat
1,2,dog
2,3,mouse


In [14]:
# the same data frame
df2 = pd.DataFrame([
    ('cat', 1),
    ('dog', 2),
    ('mouse', 3),
], columns=['animal', 'number'])

df2

,animal,number
0,cat,1
1,dog,2
2,mouse,3


## Verbs (operations) in Pandas
  
Pandas provides a "batteries-included" basic data analysis:
  - **Loading data:** `read_csv`, `read_table`, `read_sql`, and `read_html`
  - **Selection, filtering, and aggregation** (i.e. SQL-type operations): There's a special syntax for `SELECT`ing.  There's the `merge` method for `JOIN`ing.  There's also an easy syntax for what in SQL is a mouthful: Creating a new column whose value is computed from another column -- with the bonus that now the computations can use the full power of Python (though it might be faster if it didn't).
  - **"Pivot table" style aggregation:** If you're an Excel cognoscenti, you may appreciate this.
  - **NA handling:** Like R's data frames, there is good support for transforming NA values with default values / averaging tricks / etc.
  - **Basic statistics:** e.g. `mean`, `median`, `max`, `min`, and the convenient `describe`.
  - **Plugging into more advanced analytics:** Okay, this isn't batteries included.  But still, it plays reasonably with `sklearn`.
  - **Visualization:** For instance `plot` and `hist`.
  
We'll go through a little on all of these in the context of an example.

We're going to explore a dataset of mortgage insurance issued by the Federal Housing Authority (FHA).  The data is broken down by census tract and tells us how big of a player the FHA is in each tract (how many homes etc.).

## Loading data (and basic statistics / visualization)

In [24]:
names =["State_Code", "County_Code", "Census_Tract_Number",
        "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL",
        "AMT_FHA", "PCT_AMT_FHA"]

df = pd.read_csv('data/fha_by_tract.csv', names=names)  # Loading a CSV file, without a header (so we have to provide field names)


In [25]:
## Assign names to columns

names =["StateCode", "County_Code", "Census_Tract_Number",
        "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL",
        "AMT_FHA", "PCT_AMT_FHA"]

df.columns = names

In [26]:
df.columns

Index(['StateCode', 'County_Code', 'Census_Tract_Number', 'NUM_ALL', 'NUM_FHA',
       'PCT_NUM_FHA', 'AMT_ALL', 'AMT_FHA', 'PCT_AMT_FHA'],
      dtype='object')

In [27]:
df.index

RangeIndex(start=0, stop=72035, step=1)

In [28]:
df

,StateCode,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0
...,...,...,...,...,...,...,...,...,...
72030,48.0,221.0,NaN,1,0,0.0,140,0,0.0
72031,30.0,NaN,NaN,2,0,0.0,690,0,0.0
72032,36.0,47.0,558.00,10,0,0.0,3273,0,0.0
72033,30.0,31.0,8.00,21,0,0.0,5743,0,0.0


In [29]:
## Rename the columns

df = df.rename(columns={'StateCode': 'State_Code'})

In [32]:
df['State_Code'].min()

1.0

In [36]:
df['State_Code'].median()

28.0

In [37]:
df[['State_Code']].describe()

,State_Code
count,72034.000000
mean,28.322528
std,16.459507
min,1.000000
25%,13.000000
50%,28.000000
75%,42.000000
max,72.000000


In [ ]:
df.head()

In [39]:
## Create a new column from a combination of the others 'Census_Tract_Number', 'County_Code'] and 'State_Code'
df['GEOID'] = df['Census_Tract_Number']*100 + 10**6 * df['County_Code'] \
    + 10**9 * df['State_Code']   # A computed field!
df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10


In [42]:
df['result'] = df['County_Code'] + df['PCT_NUM_FHA']

In [53]:
df_describe = df.describe()

In [54]:
df_describe

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,result
count,72034.000000,71984.000000,71040.000000,72035.000000,72035.000000,72035.000000,7.203500e+04,72035.000000,72035.000000,7.104000e+04,71984.000000
mean,28.322528,85.612636,2534.598023,36.970389,9.741667,28.566878,7.886092e+03,1689.278851,29.703179,2.837319e+10,114.184140
std,16.459507,98.672445,3451.173223,53.975403,15.187832,22.404545,1.302542e+04,2800.346300,24.037779,1.648784e+10,101.768981
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,2.000000e+00,0.000000,0.000000,1.001020e+09,1.000000
25%,13.000000,29.000000,103.020000,13.000000,2.000000,11.111100,1.551000e+03,281.000000,10.780800,1.301596e+10,54.500000
50%,28.000000,63.000000,442.010000,27.000000,6.000000,25.000000,4.168000e+03,932.000000,24.753900,2.804900e+10,92.272700
75%,42.000000,109.000000,4503.012500,48.000000,13.000000,41.935500,9.668000e+03,2197.000000,44.207550,4.200348e+10,141.000000
max,72.000000,840.000000,9922.010000,9477.000000,1932.000000,100.000000,1.575871e+06,331515.000000,100.000000,7.215375e+10,880.909100


To drop a column:

In [ ]:
column_to_drop = 'GEOID'
df.drop(column_to_drop, axis = 1).head()

> Most operations produce copies (unless `inplace=True` is specified).  The `df` object still has the GEOID column.

In [ ]:
column_to_drop in df.columns

In [ ]:
# To use inplace=True is not advised. It is better to assign the transformed dataframe to a new variable

df_new = ####

print(column_to_drop in df.columns)

print(column_to_drop in df_new.columns)

> Rows can also be dropped.  Note that the indices do not reset.  The index is associated with the row, not with the order.

Note the axis !

In [ ]:
df.drop(0, axis=###).head()

By default, rows are indexed by their position.  However, any column can be made into an index:

In [ ]:
### Use State_Code as the index

Multiple levels of indexing is possible:

In [ ]:
## Use State_Code and County_Code as index


An index can be turned back into a column:

In [ ]:
## Reset index

### Basic statistics with pandas

In [ ]:
print("Percentage of mortages in each census tract insured by FHA")

df['PCT_AMT_FHA'].describe()

In [ ]:
# Apply describe to the entire dataframe


## Pandas as a plotting interface

In [ ]:
## jupyter cells admit magic functions to enhance functionalities
%matplotlib inline

In [ ]:
# plot an histogram of the column PCT_AMT_FHA

df['PCT_AMT_FHA']

The above distribution looks skewed, so let's look at its logarithm.

In [ ]:
## Transform the column to log and plot

df['LOG_AMT_ALL'] =  # Create a new column to examine

## Describe
print(df['LOG_AMT_ALL'].describe())

### And plot
df['LOG_AMT_ALL']###

# Indexing data frames

Indexing by a column name yields a data series.

In [ ]:
df['State_Code'].head()

Indexing by a list of column names gives another data frame.

In [ ]:
df[['State_Code', 'County_Code']].head()

**Question:** What will this return?

In [ ]:
type(df[['State_Code']])

In [ ]:
df[['State_Code']].head()

A data frame is an iterator that yields the column names:

In [ ]:
[col for col in df]

To select specific rows, you can try:

In [ ]:
df[:3]

To index a particular element of the frame, use the `.loc` attribute.  It takes index and column names.

In [ ]:
df.loc[:3]

In [ ]:
df.loc[3, 'State_Code']

Both can be sliced.

> Unusually for Python, both endpoints are included in the slice.

In [ ]:
df.loc[0:3, ['State_Code','Census_Tract_Number']]

Position-based indexing is available in the `.iloc` attribute.

In [ ]:
df.iloc[3, 0:3]

In [ ]:
pd.__version__

The usual slicing convention is used for `.iloc`.

In [ ]:
df.iloc[0:3, 0:3]

## Filtering data

Now the `df[...]` notation is very flexible:
  - It accepts column names (strings and lists of strings);
  - It accepts column numbers (so long as there is no ambiguity with column names);
  - It accepts _binary data series!_
  
This means that you can write
```python

 df[ df['column_name2'] == 'MD' & ( df['column_name1']==5 | df['column_name1']==6 ) ]
```   
for what you would write in SQL as

```sql
SELECT * FROM df
WHERE column_name2="MD" AND (column_name1=5 OR column_name1=6)
```           
Boolean operators on a data frame return a data series of bools.

In [ ]:
(df['State_Code'] == 1).head()

These can be combined with the (bitwise) boolean operators.  Note that, due to operator precedence, you want to wrap the individual comparisons in parentheses.

In [ ]:
((df['State_Code'] == 1) & (df['Census_Tract_Number'] == 9613)).head()

## This boolean series can be used to filter the data in pandas

In [ ]:
## Filter the data for the State_Code == 5

## Joining data

The analogue of a

```sql         
    SELECT * 
    FROM df1 INNER JOIN df2 
    ON df1.field_name=df2.field_name;
```

is

```python
    df_joined = df1.merge(df2, on='field_name')
```

You can also do left / right / outer joins, mix-and-match column names, etc.  For that consult the Pandas documentation. (The example below will do a left join.)

Of course, just looking at the distribution of insurance by census tract isn't interesting unless we know more about the census tract.

In [ ]:
df.head()

In [ ]:
# The first row is the column names, so we don't have to specify those
df_geo = pd.###('data/2013_Gaz_tracts_national.tsv', sep='\t')
df_geo.head()

In [ ]:
df_joined = df.merge(df_geo, on='GEOID', how='left')
df_joined.head()

## Aggregating data

The analog of SQL's `GROUP BY` is

    grouped = df.groupby(['field_name1', ...])...

The above is analogous to
```sql
    SELECT mean(df.value1), std(df.value2) 
    FROM df
    GROUP BY df.field_name1, ...
```
Pandas is somewhat more flexible in how you can use grouping, not requiring you to specify an aggregation function up front.  The `.groupby()` method that can later be aggregated.

In [ ]:
usps_groups = df_joined.groupby('USPS')
usps_groups

The reason Pandas doesn't require you to specify an aggregation function up front is because the groupby method by itself does little work. It returns a `DataFrameGroupBy` datatype that contains a dictionary of group keys to lists of row numbers.

In [ ]:
print(type(usps_groups.groups))
usps_groups.groups['AK'][:5]

In [ ]:
usps_groups.groups.keys()


I can retrieve the group of data associated with one key:

In [ ]:
usps_groups.get_group('AK')[:5]

Observe that this is the same as:

In [ ]:
df_joined.iloc[usps_groups.groups['AK'][:5]]

In [ ]:
usps_groups.##    # Take the mean of the rows in each group

# groupby is normally directly applied

In [ ]:
## use df_joine to calculate the sum by USPS


In [ ]:
df_by_state

You can also specify a specific aggregation function per column:

In [ ]:
usps_groups['NUM_FHA', 'NUM_ALL'].agg({'NUM_FHA': np.sum, 'NUM_ALL': np.mean}).head()

**The groupby function is especially useful when you define your own aggregation functions**

Here, we define a function that returns the row for the census track located farthest to the north. The apply function attempts to 'combine results together in an intelligent way.' The list of Series objects from each call to `farthest_north` for each USPS code is collapsed into a single DataFrame table.

In [ ]:
def farthest_north(state_df):
    # descending sort, then select row 0
    # the datatype will be a pandas Series
    return state_df.sort_values('INTPTLAT', ascending=False).iloc[0]

df_joined.groupby('USPS').apply(farthest_north)[:10]

## Sorting by indices and columns

We can sort by the row (or column) index.

In [ ]:
df_by_state.sort_index(ascending=False).head()

We can also sort by the value in a column

In [ ]:
df_by_state.sort_values('AMT_FHA', ascending=False).head()

## Unique values

As in SQL, pandas can compute unique values, value counts, and test for membership

In [ ]:
df['State_Code'].unique()[:10]

In [ ]:
df['State_Code'].value_counts().head()

In [ ]:
df[df['State_Code'].isin(df['State_Code'].head(3))]

## Handling missing and NA data


When you read in a CSV file / SQL database there are often "NA" (or "null", "None", etc.) values.

The CSV reader has a special field for specifying how this is denoted, and SQL has the built-in notion of NULL.

Note that these methods by default create a new series and do not change the original one.

For more details: http://pandas.pydata.org/pandas-docs/stable/missing_data.html

In [ ]:
df['GEOID'][:10]

`.isnull()` and `.notnull()` test for null-ness and return a Boolean series.

In [ ]:
df['GEOID'].isnull()[:10]

`.dropna()` removes the rows with null data

In [ ]:
df['GEOID'].size, df['GEOID'].dropna().size

`.fillna()` replaces N/A values with another value.  `.interpolate()` replaces null values by (linear, or quadratic, or...) interpolation.  There is support for indexing by times (not necessarily equally spaced), etc. in the documentation.

In [ ]:
df['fill_0'] = df['GEOID'].fillna(0)                          # Fills constant value, here 0
df['fill_forward'] = df['GEOID'].fillna(method='ffill')       # Fill forwards
df['fill_back'] = df['GEOID'].fillna(method='bfill', limit=5) # Fill backwards, at most 5
df['fill_mean'] = df['GEOID'].fillna(df['GEOID'].mean())      # Fills constant value, here the mean (imputation)
df['fill_interp'] = df['GEOID'].interpolate()                 # Fills interpolated value

df[['GEOID', 'fill_0', 'fill_forward', 'fill_back', 'fill_mean', 'fill_interp']][:10]

### Note
N/A values are (usually) smartly ignored when performing other calculations on dataframes. For example, when using string methods on series:

Applying mean on numeric data ignores NA's by default (check docs):

In [ ]:
df['GEOID'].mean()

## Manipulating strings

Element-wise string operations are available through the `.str` attribute.

In [ ]:
states = df_joined['USPS'].dropna()
states[states.str.contains('A')].head()

## Function application and mapping

For element-wise function application, the most straightforward thing to do is to apply numpy functions to these objects:

In [ ]:
## Create a matrix of 4x6 and calculate its sin

df1 = pd.DataFrame(np.arange(24).reshape(##,##))

##sin 

This relies on numpy functions automatically broadcasting themselves to work element-wise.  To apply a pure-python function to each element, use the `.applymap()` method.

In [ ]:
df1.applymap(lambda x: 2*x)

However, sometimes you want to compute things column-wise or row-wise.  In this case, you will need to use the `apply` method. 

For example, the following takes the range of each column.

In [ ]:
df1.apply(lambda x: x.max() - x.min())

And this takes the range of reach row.

In [ ]:
df1.apply(lambda x: x.max() - x.min(), axis=1)

## Pivot tables

Data frames can contain multiple indices for rows or columns.  For example, grouping by two columns will produce a two-level row index.

This can be done with one step with the `pivot_table()` function.

In [ ]:
pd.pivot_table(df, index='State_Code', columns='County_Code',
               values='NUM_ALL', aggfunc=np.sum).head()

You may already by familiar with pivot tables in Excel.  These work similarly, and area  good tool for changing the dependent and independent variables for aggregations of data. See http://pandas.pydata.org/pandas-docs/stable/reshaping.html for more information.

## Plugging into more advanced analytics

Almost any "advanced analytics" tool in the Python ecosystem is going to take as input `np.array` type arrays.  You can access the underlying array of a data frame column as

        df['column'].values
        
Many of them take `nd.array` whose underlying data can be accessed by 

        df.values
        
directly.  *Most* of the time, they will take `df['column']` and `df` without needing to look at values.

This is particularly important if you want to use Pandas with the sklearn library. See this [blog post](http://www.markhneedham.com/blog/2013/11/09/python-making-scikit-learn-and-pandas-play-nice/) for an example.